In [2]:
from textblob import TextBlob as tb
import os
import pandas as pd
import numpy as np

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
#global containers
features = []
features_infected = []
features_non_infected = []

In [5]:
stopwords = set(w.rstrip() for w in open('stopwords.txt'))

In [6]:
def build_features(file_name):
    list_row = []   
    data_df = pd.read_csv(file_name)
    print("file reading complete")
    data_df.drop (columns=data_df. columns [0], axis= 1, inplace= True)
    print(data_df.head())
    for i, r in data_df.iterrows():
        title = r['title']
        litigation = r['lt']      
        litigation_combined = str(title) + str(litigation)
        tbb  = tb(litigation_combined)
        litigation_combined = tbb.words
        
        row =[]
        for i in range(0,len(litigation_combined)):
            if len(litigation_combined[i]) > 2 and  len(litigation_combined[i]) < 11 and litigation_combined[i].lower() not in stopwords:
                val = str(litigation_combined[i])
                #val ="%r"%val #converting into raw string
                if val.isdigit():
                    val = str(len(val)) + "digit"
                row.append(val)
                #print("row")
        list_row.append(row)   
    return list_row

In [7]:
filename = "infected.csv"
features_infected = features_infected + build_features(filename)

file reading complete
   lt_no    yr title                                                 lt  class
0  14775  1996   NaN   Washington, D C   14775January 4, 1996V  LIND...      1
1  14776  1996   NaN    14776January 4, 1996 Accounting and Auditing...      1
2  14799  1996   NaN  U S  Securities and Exchange Commission 14799J...      1
3  14810  1996   NaN     14810February 2, 1996SECURITIESAND EXCHANGE...      1
4  14811  1996   NaN   Securities and Exchange Commission  14811Febr...      1


In [8]:
len(features_infected)

1373

In [9]:
filename = "non_infected.csv"
features_non_infected = features_non_infected + build_features(filename)

file reading complete
   lt_no    yr title                                                 lt  class
0  14770  1996   NaN   Washington, D C   14770January 3, 1996ACCOUNT...      0
1  14771  1996   NaN  v  WILLIAM J  YOUNG, UnitedStates District Cou...      0
2  14772  1996   NaN    14772January 3, 1995SEC v  Basic Energy & Af...      0
3  14773  1996   NaN    14773January 3, 1996SEC v  Edmund J  Lopinsk...      0
4  14774  1996   NaN   Washington, D C   14774January 4, 1996V  ROBE...      0


In [10]:
len(features_non_infected)

7840

In [11]:
# feature to index mapping
feature_index_map = {}
counter = 0

In [12]:
for feature_row in features_infected:
    for feature in feature_row:
        if feature not in feature_index_map:
            feature_index_map[feature] = counter
            counter = counter + 1
            
for feature_row in features_non_infected:
    for feature in feature_row:
        if feature not in feature_index_map:
            feature_index_map[feature] = counter
            counter = counter + 1

In [13]:
# feature vector
def features_to_vector(features, cls):
    row = np.zeros(len(feature_index_map) +1 ) # last column is for the class attribute, 1= infected, 2 = noninfected
    for f in features:
        index = feature_index_map[f]
        row[index] = 1
        #@todo normalization, if that helps 
        row[-1] = cls
    return row

In [14]:
data_feature_vector = np.zeros(((len(features_infected) + len(features_non_infected)), (len(feature_index_map) + 1)))

In [15]:
row_count =0

In [16]:
for feature_row in features_infected:
    vector = features_to_vector(feature_row, 1) # 1 =  infected
    data_feature_vector[row_count, :] = vector
    row_count = row_count + 1

In [17]:
for feature_row in features_non_infected:
    vector = features_to_vector(feature_row, 0) # o = non infected
    data_feature_vector[row_count, :] = vector
    row_count = row_count + 1

In [18]:
data_feature_vector_df = pd.DataFrame(data_feature_vector)

In [19]:
len(feature_index_map)

65761

In [20]:
data_feature_vector_df.shape

(9213, 65762)

In [21]:
data_feature_vector_df.head()

,0,1,2,3,4,5,6,7,8,9,...,65752,65753,65754,65755,65756,65757,65758,65759,65760,65761
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
data_feature_vector_df.to_csv("feature_vector.csv",sep=',')
print(data_feature_vector_df[:5])

In [ ]:
feature_index_map_list = []
for key, val in feature_index_map.items():
    temp = [val,key]
    feature_index_map_list.append(temp)

In [ ]:
temp = [len(feature_index_map_list), "class"]
feature_index_map_list.append(temp)

In [ ]:
feature_index_map_df = pd.DataFrame(feature_index_map_list, columns = ['idx', 'feature'])

In [ ]:
feature_index_map_df.to_csv("feature_vector_index_map.csv", sep=',', index = False)